Ход решения задачи:  
для двух кубиков

In [1]:
num_cubs = 2
from itertools import product
outcomes = range(1, 7)  # возможные исходы для одного кубика (1-6)
combs = product(outcomes, repeat=num_cubs)  # все возможные комбинации бросков

In [2]:
for i, e in enumerate(combs):
    print(e, end='')
    if (i+1) % 6 == 0:
        print()

(1, 1)(1, 2)(1, 3)(1, 4)(1, 5)(1, 6)
(2, 1)(2, 2)(2, 3)(2, 4)(2, 5)(2, 6)
(3, 1)(3, 2)(3, 3)(3, 4)(3, 5)(3, 6)
(4, 1)(4, 2)(4, 3)(4, 4)(4, 5)(4, 6)
(5, 1)(5, 2)(5, 3)(5, 4)(5, 5)(5, 6)
(6, 1)(6, 2)(6, 3)(6, 4)(6, 5)(6, 6)


In [3]:
from collections import defaultdict
total_possibilities = 6 ** num_cubs
hit_counts = defaultdict(int)

In [4]:
(16  + 11) % 12 + 1  # учтем цикл на 12й ячейке

4

In [5]:
combs = product(outcomes, repeat=num_cubs)

for i, comb in enumerate(combs):
    total = sum(comb)
    cell = (total + 11) % 12 + 1  # учтем цикл на 12й ячейке
    print((total, cell), end="")
    if (i+1) % 6 == 0:
        print()
    hit_counts[cell] += 1

hit_counts

(2, 2)(3, 3)(4, 4)(5, 5)(6, 6)(7, 7)
(3, 3)(4, 4)(5, 5)(6, 6)(7, 7)(8, 8)
(4, 4)(5, 5)(6, 6)(7, 7)(8, 8)(9, 9)
(5, 5)(6, 6)(7, 7)(8, 8)(9, 9)(10, 10)
(6, 6)(7, 7)(8, 8)(9, 9)(10, 10)(11, 11)
(7, 7)(8, 8)(9, 9)(10, 10)(11, 11)(12, 12)


defaultdict(int,
            {2: 1,
             3: 2,
             4: 3,
             5: 4,
             6: 5,
             7: 6,
             8: 5,
             9: 4,
             10: 3,
             11: 2,
             12: 1})

In [6]:
# вычислим вероятности
probs = {cell: count / total_possibilities for cell, count in hit_counts.items()}
probs

{2: 0.027777777777777776,
 3: 0.05555555555555555,
 4: 0.08333333333333333,
 5: 0.1111111111111111,
 6: 0.1388888888888889,
 7: 0.16666666666666666,
 8: 0.1388888888888889,
 9: 0.1111111111111111,
 10: 0.08333333333333333,
 11: 0.05555555555555555,
 12: 0.027777777777777776}

In [7]:
1/36

0.027777777777777776

In [8]:
# если кудато не встали
for i in range(1, 13):
    if i not in probs:
        probs[i] = 0

In [9]:
import pandas as pd

def get_probs(num_dice):
    outcomes = range(1, 7) 
    combs = product(outcomes, repeat=num_dice) 

    total_possibilities = 6 ** num_dice
    hit_counts = defaultdict(int)

    for comb in combs:
        total = sum(comb)
        cell = (total + 11) % 12 + 1 
        hit_counts[cell] += 1

    probs = {cell: count / total_possibilities for cell, count in hit_counts.items()}

    for i in range(1, 13):
        if i not in probs:
            probs[i] = 0

    return probs


probs_2 = get_probs(2)
probs_3 = get_probs(3)

# создание таблицы pandas
df = pd.DataFrame({
    '2_cubs': [probs_2[i] for i in range(1, 13)],
    '3_cubs': [probs_3[i] for i in range(1, 13)],
}, index=[f"Cell {i}" for i in range(1, 13)])

df

2_cubs    3_cubs
Cell 1   0.000000  0.097222
Cell 2   0.027778  0.069444
Cell 3   0.055556  0.050926
Cell 4   0.083333  0.041667
Cell 5   0.111111  0.041667
Cell 6   0.138889  0.050926
Cell 7   0.166667  0.069444
Cell 8   0.138889  0.097222
Cell 9   0.111111  0.115741
Cell 10  0.083333  0.125000
Cell 11  0.055556  0.125000
Cell 12  0.027778  0.115741

In [10]:
probs_6 = get_probs(6)
df = pd.DataFrame({
    '2_cubs': [probs_2[i] for i in range(1, 13)],
    '3_cubs': [probs_3[i] for i in range(1, 13)],
    '6_cubs': [probs_6[i] for i in range(1, 13)],
}, index=[f"Cell {i}" for i in range(1, 13)])

df

2_cubs    3_cubs    6_cubs
Cell 1   0.000000  0.097222  0.077418
Cell 2   0.027778  0.069444  0.073045
Cell 3   0.055556  0.050926  0.071416
Cell 4   0.083333  0.041667  0.073045
Cell 5   0.111111  0.041667  0.077418
Cell 6   0.138889  0.050926  0.083333
Cell 7   0.166667  0.069444  0.089249
Cell 8   0.138889  0.097222  0.093621
Cell 9   0.111111  0.115741  0.095250
Cell 10  0.083333  0.125000  0.093621
Cell 11  0.055556  0.125000  0.089249
Cell 12  0.027778  0.115741  0.083333

для 12 долго считает, используем аналитическую формулу

In [11]:
from functools import lru_cache

@lru_cache(maxsize=None)

def prob_sum(s, n):
    if n == 1:
        return 1 if 1 <= s <= 6 else 0
    if s < n or s > 6 * n:  # мин  макс возможных сумм для n кубиков
        return 0
    
    # рекурсия
    return sum(prob_sum(s-k, n-1) for k in range(1, 7))

In [12]:
prob_sum(12, 2), prob_sum(2, 2), prob_sum(5, 2), prob_sum(1, 2)
# (1+4, 2+3, 3+2, 4+1)

(1, 1, 4, 0)

In [13]:
n = 3
prob_sum(6*n, n), prob_sum(3, n), prob_sum(5, n), prob_sum(1, n)
#(1+1+3, 1+3+1, 3+1+1, 1+2+2, 2+1+2, 2+2+1)

(1, 1, 6, 0)

In [14]:
def get_probs_optim(num_dice):
    total_possibilities = 6 ** num_dice
    probs = {}

    for s in range(num_dice, 6*num_dice + 1):
        cell = (s + 11) % 12 + 1
        probs[cell] = probs.get(cell, 0) + prob_sum(s, num_dice) / total_possibilities

    for i in range(1, 13):
        if i not in probs:
            probs[i] = 0

    return probs

In [15]:
probs_2_optim = get_probs_optim(2)
probs_3_optim = get_probs_optim(3)
probs_6_optim = get_probs_optim(6)

In [16]:
df_opti = pd.DataFrame({
    '2_cubs': [probs_2[i] for i in range(1, 13)],
    '2_cubs_opt': [probs_2_optim[i] for i in range(1, 13)],
    '3_cubs': [probs_3[i] for i in range(1, 13)],
    '3_cubs_opt': [probs_3_optim[i] for i in range(1, 13)],
    '6_cubs': [probs_6[i] for i in range(1, 13)],
    '6_cubs_opt': [probs_6_optim[i] for i in range(1, 13)],

}, index=[f"Cell {i}" for i in range(1, 13)])

df_opti

2_cubs  2_cubs_opt    3_cubs  3_cubs_opt    6_cubs  6_cubs_opt
Cell 1   0.000000    0.000000  0.097222    0.097222  0.077418    0.077418
Cell 2   0.027778    0.027778  0.069444    0.069444  0.073045    0.073045
Cell 3   0.055556    0.055556  0.050926    0.050926  0.071416    0.071416
Cell 4   0.083333    0.083333  0.041667    0.041667  0.073045    0.073045
Cell 5   0.111111    0.111111  0.041667    0.041667  0.077418    0.077418
Cell 6   0.138889    0.138889  0.050926    0.050926  0.083333    0.083333
Cell 7   0.166667    0.166667  0.069444    0.069444  0.089249    0.089249
Cell 8   0.138889    0.138889  0.097222    0.097222  0.093621    0.093621
Cell 9   0.111111    0.111111  0.115741    0.115741  0.095250    0.095250
Cell 10  0.083333    0.083333  0.125000    0.125000  0.093621    0.093621
Cell 11  0.055556    0.055556  0.125000    0.125000  0.089249    0.089249
Cell 12  0.027778    0.027778  0.115741    0.115741  0.083333    0.083333

In [17]:
df_opti['2_cubs_opt'] - df_opti['2_cubs_opt']

Cell 1     0.0
Cell 2     0.0
Cell 3     0.0
Cell 4     0.0
Cell 5     0.0
Cell 6     0.0
Cell 7     0.0
Cell 8     0.0
Cell 9     0.0
Cell 10    0.0
Cell 11    0.0
Cell 12    0.0
Name: 2_cubs_opt, dtype: float64

In [18]:
for i in [2,3,6]:
    df_opti[f'diff_{i}'] = df_opti[f'{i}_cubs'] - df_opti[f'{i}_cubs']

In [19]:
df_opti

2_cubs  2_cubs_opt    3_cubs  3_cubs_opt    6_cubs  6_cubs_opt  \
Cell 1   0.000000    0.000000  0.097222    0.097222  0.077418    0.077418   
Cell 2   0.027778    0.027778  0.069444    0.069444  0.073045    0.073045   
Cell 3   0.055556    0.055556  0.050926    0.050926  0.071416    0.071416   
Cell 4   0.083333    0.083333  0.041667    0.041667  0.073045    0.073045   
Cell 5   0.111111    0.111111  0.041667    0.041667  0.077418    0.077418   
Cell 6   0.138889    0.138889  0.050926    0.050926  0.083333    0.083333   
Cell 7   0.166667    0.166667  0.069444    0.069444  0.089249    0.089249   
Cell 8   0.138889    0.138889  0.097222    0.097222  0.093621    0.093621   
Cell 9   0.111111    0.111111  0.115741    0.115741  0.095250    0.095250   
Cell 10  0.083333    0.083333  0.125000    0.125000  0.093621    0.093621   
Cell 11  0.055556    0.055556  0.125000    0.125000  0.089249    0.089249   
Cell 12  0.027778    0.027778  0.115741    0.115741  0.083333    0.083333   

         diff_2  diff_3  diff_6  
Cell 1      0.0     0.0     0.0  
Cell 2      0.0     0.0     0.0  
Cell 3      0.0     0.0     0.0  
Cell 4      0.0     0.0     0.0  
Cell 5      0.0     0.0     0.0  
Cell 6      0.0     0.0     0.0  
Cell 7      0.0     0.0     0.0  
Cell 8      0.0     0.0     0.0  
Cell 9      0.0     0.0     0.0  
Cell 10     0.0     0.0     0.0  
Cell 11     0.0     0.0     0.0  
Cell 12     0.0     0.0     0.0

In [20]:
probs_12_optim = get_probs_optim(12)

In [21]:
df = pd.DataFrame({

    '3_cubs': [probs_3[i] for i in range(1, 13)],
    '6_cubs': [probs_6[i] for i in range(1, 13)],
    '12_cubs': [probs_12_optim[i] for i in range(1, 13)],
}, index=[f"Cell {i}" for i in range(1, 13)])

df

3_cubs    6_cubs   12_cubs
Cell 1   0.097222  0.077418  0.082599
Cell 2   0.069444  0.073045  0.082910
Cell 3   0.050926  0.071416  0.083333
Cell 4   0.041667  0.073045  0.083757
Cell 5   0.041667  0.077418  0.084067
Cell 6   0.050926  0.083333  0.084181
Cell 7   0.069444  0.089249  0.084067
Cell 8   0.097222  0.093621  0.083757
Cell 9   0.115741  0.095250  0.083333
Cell 10  0.125000  0.093621  0.082910
Cell 11  0.125000  0.089249  0.082599
Cell 12  0.115741  0.083333  0.082486